In [5]:
import cv2
import mediapipe as mp
import numpy as np
import onnxruntime as ort
import json
import time
from collections import deque, Counter

print("✅ Đã import thư viện thành công.")

✅ Đã import thư viện thành công.


In [ ]:
class WebStyleGestureRecognizer:
    def __init__(self, model_path="sign_model.onnx", meta_path="model_meta.json"):
        # --- CẤU HÌNH GIỐNG MAIN.JS ---
        self.SEQ_LEN = 30
        self.FEAT_DIM = 63
        self.VOTE_WIN = 15       # Số frame dùng để vote (Smoothing)
        self.CONF_THRESH = 0.60  # Ngưỡng tin cậy
        self.PRED_STRIDE = 2     # Cứ 2 frame mới predict 1 lần để tối ưu FPS

        # State buffers
        self.seq_buffer = deque(maxlen=self.SEQ_LEN) # Buffer đầu vào cho model
        self.pred_history = deque(maxlen=self.VOTE_WIN) # Buffer lưu kết quả để vote
        self.frame_counter = 0

        # Load Labels
        try:
            with open(meta_path, "r") as f:
                self.meta = json.load(f)
            self.labels = self.meta["labels"]
            print(f"✅ Labels loaded: {len(self.labels)} classes")
        except:
            print("⚠️ Không tìm thấy file json, dùng labels mặc định.")
            self.labels = []

        # Load ONNX Model
        try:
            self.session = ort.InferenceSession(model_path)
            self.input_name = self.session.get_inputs()[0].name
            print("✅ Model loaded successfully!")
        except Exception as e:
            print(f"❌ Lỗi load model: {e}")
            self.session = None

    # --- TƯƠNG ĐƯƠNG HÀM extractFeat63 TRONG MAIN.JS ---
    def extract_features(self, landmarks):
        """
        Input: MediaPipe landmarks object
        Output: List 63 float (Normalized by Wrist & Scale)
        """
        if not landmarks:
            return [0.0] * self.FEAT_DIM

        # Convert to list of dicts for easier access usually,
        # but here we access direct attributes .x .y .z
        lms = landmarks.landmark
        wrist = lms[0]

        # Tính Scale: Khoảng cách giữa Wrist(0) và Middle_MCP(9)
        # JS: const dx = landmarks[9].x - wrist.x; ...
        middle_mcp = lms[9]
        dx = middle_mcp.x - wrist.x
        dy = middle_mcp.y - wrist.y
        dz = middle_mcp.z - wrist.z
        scale = np.sqrt(dx*dx + dy*dy + dz*dz) or 1.0

        features = []
        for lm in lms:
            # Normalize relative to wrist and scale
            features.append((lm.x - wrist.x) / scale)
            features.append((lm.y - wrist.y) / scale)
            features.append((lm.z - wrist.z) / scale)

        return features

    # --- TƯƠNG ĐƯƠNG HÀM majorityVote TRONG MAIN.JS ---
    def get_majority_vote(self):
        if not self.pred_history:
            return None
        # Đếm tần xuất xuất hiện của các index trong history
        count = Counter(self.pred_history)
        # Lấy phần tử xuất hiện nhiều nhất
        most_common = count.most_common(1)[0] # Trả về (class_idx, frequency)
        return most_common[0]

    # --- LOGIC CHÍNH: classifyBySequence ---
    def process(self, landmarks):
        if self.session is None:
            return None, 0.0

        # 1. Feature Extraction & Buffer Update
        # Nếu không có landmarks, push vector 0 (QUAN TRỌNG ĐỂ GIỐNG JS)
        feats = self.extract_features(landmarks)
        self.seq_buffer.append(feats)

        # Chưa đủ dữ liệu thì chưa chạy
        if len(self.seq_buffer) < self.SEQ_LEN:
            return None, 0.0

        # 2. Stride Check (Tối ưu hiệu năng)
        self.frame_counter += 1
        if self.frame_counter % self.PRED_STRIDE != 0:
            return None, 0.0 # Bỏ qua frame này, giữ kết quả cũ nếu cần

        # 3. Inference
        input_tensor = np.array([self.seq_buffer], dtype=np.float32) # Shape (1, 30, 63)
        try:
            outputs = self.session.run(None, {self.input_name: input_tensor})
            logits = outputs[0][0] # Raw scores

            # JS code dùng raw logits để so sánh max
            best_idx = np.argmax(logits)
            conf = logits[best_idx] # Raw confidence

            # 4. Smoothing (Majority Vote)
            self.pred_history.append(best_idx)
            stable_idx = self.get_majority_vote()

            # Lấy label và conf của class ổn định nhất
            final_label = self.labels[stable_idx]

            # *Lưu ý*: Conf trả về ở đây là của frame hiện tại,
            # nhưng label là của vote.

            return final_label, float(conf)

        except Exception as e:
            print(f"Inference Error: {e}")
            return None, 0.0

In [ ]:
# Cấu hình MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.6, # Giống JS config
    min_tracking_confidence=0.5
)

# Khởi tạo Engine
engine = WebStyleGestureRecognizer()

# Mở Camera
cap = cv2.VideoCapture(0)

# Biến lưu kết quả hiển thị (để tránh nhấp nháy khi Stride bỏ qua frame)
display_label = "WAITING..."
display_conf = 0.0
bar_width = 0

try:
    while cap.isOpened():
        success, image = cap.read()
        if not success: continue

        # 1. Xử lý ảnh
        image = cv2.flip(image, 1) # Lật gương
        h, w, _ = image.shape
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 2. Detect Hands
        results = hands.process(image_rgb)

        landmarks = None
        if results.multi_hand_landmarks:
            landmarks = results.multi_hand_landmarks[0] # Lấy tay đầu tiên

            # Vẽ Skeleton (Giống style.css màu xanh neon)
            mp_drawing.draw_landmarks(
                image,
                landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

        # 3. Gọi Engine (Logic giống main.js)
        # Lưu ý: Ngay cả khi landmarks là None, ta vẫn gọi process
        # để nó đẩy vector 0 vào buffer (cơ chế continuous)
        label, conf = engine.process(landmarks)

        # Update UI state nếu có kết quả mới (do stride skip frame)
        if label is not None:
            # Giả lập logic hiển thị confidence bar của JS
            # JS: Math.min(Math.max(pred.conf + 5, 0) / 10, 1) * 100;
            # Vì model output raw logit (có thể âm), công thức này giúp scale lên %
            normalized_conf = min(max(conf + 5, 0) / 10, 1)

            if normalized_conf * 10 > engine.CONF_THRESH * 10: # Check ngưỡng
                display_label = label
                display_conf = normalized_conf
                bar_width = int(normalized_conf * 200) # Max width 200px
            else:
                # Nếu conf thấp thì làm mờ hoặc hiện chữ chờ
                display_label = label
                bar_width = int(normalized_conf * 200)

        # 4. Vẽ UI mô phỏng Web (Overlay)

        # Box nền chứa Label
        cv2.rectangle(image, (w//2 - 150, 40), (w//2 + 150, 140), (0, 0, 0), -1) # Nền đen mờ

        # Label text (Màu xanh cyan giống #00ffcc)
        text_color = (204, 255, 0) if display_conf > engine.CONF_THRESH else (100, 100, 100)
        font_scale = 1.5
        text_size = cv2.getTextSize(display_label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 3)[0]
        text_x = (w - text_size[0]) // 2

        cv2.putText(image, display_label, (text_x, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, 3, cv2.LINE_AA)

        # Confidence Bar (Thanh màu xanh dưới chữ)
        bar_x = (w - 200) // 2
        bar_y = 115
        # Vẽ khung bar background
        cv2.rectangle(image, (bar_x, bar_y), (bar_x + 200, bar_y + 10), (100, 100, 100), -1)
        # Vẽ bar fill
        fill_width = max(0, min(200, bar_width))
        cv2.rectangle(image, (bar_x, bar_y), (bar_x + fill_width, bar_y + 10), (204, 255, 0), -1)

        # Show
        cv2.imshow('Web Replica Inference', image)
        if cv2.waitKey(1) & 0xFF == 27: # ESC để thoát
            break

except KeyboardInterrupt:
    print("Stopped.")
finally:
    cap.release()
    cv2.destroyAllWindows()

✅ Labels loaded: 27 classes
✅ Model loaded successfully!


I0000 00:00:1766583002.338674   71490 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1766583002.340343   71684 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: AMD Radeon 780M (radeonsi, phoenix, LLVM 20.1.2, DRM 3.61, 6.14.0-37-generic)
W0000 00:00:1766583002.364999   71679 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766583002.383110   71680 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/dangkhoi/miniconda3/envs/env1/lib/python3.11/site-packages/cv2/qt/plugins"
W0000 00:00:1766583003.749380   71676 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DI